In [2]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import random as rand
import queue
import csv
from collections import OrderedDict
from IPython.display import clear_output
import csv
from heapq import merge
from sklearn import preprocessing

from sklearn.datasets import make_circles, make_moons
import matplotlib.pyplot as plt
import matplotlib
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from IPython.core.debugger import set_trace
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable

In [17]:
rand.seed(37)
AAMass = {'A':71.037114, 'C':103.009185, 'D':115.026943, 'E':129.042593, 'F':147.068414, 'G':57.021464, 'H':137.058912,
          'I':113.084064, 'K':128.094963, 'L':113.084064, 'M':131.040485, 'N':114.042927, 'P':97.052764, 'Q':128.058578,
          'R':156.101111, 'S':87.032028, 'T':101.047679, 'V':99.068414, 'W':186.079313, 'Y':163.0633}

H2O = 18.015
NH3 = 17.031
PROTON = 1.00727647
specsize = 8000
charge = 2
use_mods = False

In [18]:
def GetAAMass(AA):
    return AAMass[AA] + 57.021464 if AA == 'C' else AAMass[AA]

def GetSpectrum(Seq):
    size = len(Seq)
    outsize = 2*size
    bspectrum = []
    bspectrumaux = []
    yspectrum = []
    yspectrumaux = []
    
    bspectrum.append(GetAAMass(Seq[0]) + PROTON)
    yspectrum.append(GetAAMass(Seq[-1]) + H2O + PROTON)
    
    for i, (fAA, bAA) in enumerate(zip((Seq[1:]), Seq[-2::-1])):
        bspectrum.append(bspectrum[i] + GetAAMass(fAA))
        yspectrum.append(yspectrum[i] + GetAAMass(bAA))
    
#    bspectrumaux = [(bion + PROTON) / 2 for bion in bspectrum]
#    bspectrumaux.extend([bion - H2O for bion in bspectrum])
#    bspectrumaux.extend([bion - NH3 for bion in bspectrum])
    
#    yspectrumaux = [(yion + PROTON) / 2 for yion in yspectrum]
#    yspectrumaux.extend([yion - H2O for yion in yspectrum])
#    yspectrumaux.extend([yion - NH3 for yion in yspectrum])
    
#    bspectrum.extend(bspectrumaux)
#    yspectrum.extend(yspectrumaux)
    
#     bspectrum.sort()
#     yspectrum.sort()
    
    mergedout = list(merge(bspectrum, yspectrum))
    if mergedout[-1] > specsize:
        print(mergedout[-1])
        print(Seq)
    tspec = np.zeros(specsize)
    tspec[np.rint(mergedout).astype(int)] = 1
    return tspec

by = GetSpectrum('ACDEFG')
print('printing by:\n' + str(by))
print(by.shape)
print(AAMass['L'])

printing by:
[0. 0. 0. ... 0. 0. 0.]
(8000,)
113.084064


In [19]:
def reset_signals():
    isName = isMW = isNumPeaks = False

In [20]:
f = open("human_consensus_final_true_lib.msp", "r")
lines = f.readlines()
newcontents = []
specid = 0
prev = 0
num_specs = 0
for i, line in enumerate(lines):
    if line.startswith('Name') and '_' in line:
        l_charge = int(line[line.find('_') - 1])
        if l_charge != charge:
            continue
        if use_mods:
            num_specs += 1
        elif '(' not in line and ')' not in line:
            num_specs += 1
        new = int((i/len(lines)) * 100)
        if (new > prev):
            clear_output(wait=True)
            print(str(new) + '%')
            prev = new
f.close()
print(type(lines))
print('len of file: ' + str(len(lines)))
print(num_specs)

99%
<class 'list'>
len of file: 50017570
148402


#### Non Mods:
Number of charge 1 examples: 18230  
Number of charge 2 examples: 130172  
Number of charge 3 examples: 70741  
  
#### Con Mods:
Number of charge 2 examples: 184994

In [21]:
if not lines:
    f=open("human_consensus_final_true_lib.msp", "r")
#fo = open('output.csv', 'w')
#fo.write('Q,P,N\n')
fixedlen = 300
dataset = []
label = []
lines = f.readlines() if not lines else lines
f.close()
print('len of file: ' + str(len(lines)))
count = 0
limit = 200000
pep = mass = numPeaks = 0
#spec = []
isName = isMW = isNumPeaks = False
new = prev = 0
maxpeaks = maxmoz = 0
i = 0
while i < len(lines) and limit > 0:
    line = lines[i]
    i += 1
    splits = line.split(':') 
    if (splits[0] == 'Name') and '_' in line:
        split1 = splits[1]
        l_charge = int(split1[split1.find('_') - 1])
        if l_charge != charge: # l_charge == l_charge always true.
            continue
        if use_mods:
            pep = split1.split('/')[0].lstrip(' ')
            isName = True
        elif '(' not in splits[1] and ')' not in splits[1]:
            pep = split1.split('/')[0].lstrip(' ')
            isName = True
        
    if (isName and splits[0] == 'MW'):
        mass = float(splits[1])
        if round(mass) < specsize:
            isMW = True
            #limit = limit - 1
        else:
            isName = isMW = isNumPeaks = False
            continue
        
    if (isName and isMW and splits[0] == 'Num peaks'):
        numPeaks = int(splits[1])
        if numPeaks > maxpeaks:
            maxpeaks = numPeaks
            
        spec = np.zeros(specsize)
        while (lines[i] != '\n'):
            mzline = lines[i]
            i +=1
            mzsplits = mzline.split('\t')
            moz, intensity = float(mzsplits[0]), float(mzsplits[1])
            if moz > maxmoz:
                maxmoz = moz
            spec[round(moz)] += round(intensity)
        
        #padlen = fixedlen - len(spec)
        #leftpad = rand.randint(0, padlen)
        #rightpad = padlen - leftpad
        #npspec = np.reshape(spec, (len(spec), 2))
        #npspec = np.pad(npspec, ((leftpad, rightpad), (0, 0)), 'constant', constant_values=(0, 0))
        spec = np.clip(spec, None, 1000.0)
        spec = preprocessing.scale(spec)
        #min_max_scaler = sklearn.preprocessing.MinMaxScaler()
        #npspec[:,1] = sklearn.preprocessing.minmax_scale(npspec[:,1])
        
        isNumPeaks = True
        
    if isName and isMW and isNumPeaks:
        isName = isMW = isNumPeaks = False
        revPep = pep[0] + pep[1:-1][::-1] + pep[-1]
        if pep == revPep:
            print('decoy is the same. shuffling')
            revPep = ''.join(rand.sample(revPep,len(revPep)))
        #fo.write('"' + str(list(spec)).strip('[]') + '",' + str(pep) + ',' + str(revPep) + '\n')
        tspec = preprocessing.scale(GetSpectrum(pep))
        rtspec = preprocessing.scale(GetSpectrum(revPep))
        #padlen = fixedlen - len(tspec)
        #leftpad = rand.randint(0, padlen)
        #rightpad = padlen - leftpad
        #tspec = np.pad(tspec, ((leftpad, rightpad), (0, 0)), 'constant', constant_values=(0, 0))
        #padlen = fixedlen - len(rtspec)
        #leftpad = rand.randint(0, padlen)
        #rightpad = padlen - leftpad
        #rtspec = np.pad(rtspec, ((leftpad, rightpad), (0, 0)), 'constant', constant_values=(0, 0))
        
        #spectensor = transforms.ToTensor()(spec).view(specsize,-1).float()
        #tspectensor = transforms.ToTensor()(tspec).view(specsize,-1).float()
        #rtspectensor = transforms.ToTensor()(rtspec).view(specsize,-1).float()
        
        dataset.append([spec, tspec, rtspec])
        label.append([1, -1])
        #dataset.append([np.reshape(spec, (len(spec), 2)), tspec, rtspec])
        count = count + 1
        pep = mass = numPeaks = 0
        spec = []
        new = int((i/len(lines)) * 100)
        if (new > prev):
            clear_output(wait=True)
            print(str(new) + '%')
            prev = new

print('max peaks: ' + str(maxpeaks))
print('count: ' + str(count))
print('max moz: ' + str(maxmoz))
#fo.close()

99%
decoy is the same. shuffling
decoy is the same. shuffling
decoy is the same. shuffling
max peaks: 251
count: 130172
max moz: 2616.7


In [ ]:
print(len(dataset))
print(len(label))

#dataset[:,0] = preprocessing.scale(dataset[:,0])

#transformer = transforms.Compose([transforms.ToTensor(),
#                                 transforms.Normalize((0.5,),(1.0,))])

#dataset = transformer(dataset)
print('splitting...')
tmpTrainData, tmpTestData = train_test_split(
            dataset, test_size = 0.2, random_state = rand.randint(0, 1000), shuffle = True)

tmpTestData = np.asarray(tmpTestData)
tmpTestOut = np.append(np.ones(len(tmpTestData)), np.zeros(len(tmpTestData)))
tmpTestData = np.concatenate((tmpTestData[:,[0,1]], tmpTestData[:,[0,2]]), axis=0)

# XTrainTensor = transformer(tmpTrainData, dtype=torch.float)
# XTestTensor = transformer(tmpTestData, dtype=torch.float)
# yTrainTensor = transformer(tmpTrainLbl, dtype=torch.long)
# yTestTensor = transformer(tmpTestLbl, dtype=torch.long)

print('almost done.')
XTrainTensor = torch.tensor(tmpTrainData, dtype=torch.float)
XTestTensor = torch.tensor(tmpTestData, dtype=torch.float)
yTestTensor = torch.tensor(tmpTestOut, dtype=torch.float)

testDataset = torch.utils.data.TensorDataset(XTestTensor, yTestTensor)
print('done')

In [61]:
batch_size = 1024
do_learn = True
save_frequency = 2
lr = 0.00001
num_epochs = 200
weight_decay = 0.0001
margin = 0.2
#torch.manual_seed(0)
#torch.cuda.manual_seed(0)
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda')

train_loader = torch.utils.data.DataLoader(dataset=XTrainTensor,
                                         batch_size=batch_size,
                                         shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=testDataset,
                                         batch_size=batch_size,
                                         shuffle=True)
#torch.save(train_loader, 'train_loader.pt')
#torch.save(test_loader, 'test_loader.pt')

In [63]:
#train_loader = torch.load('train_loader.pt')
#test_loader = torch.load('test_loader.pt')
print(testDataset.shape)

AttributeError: 'TensorDataset' object has no attribute 'shape'

In [50]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        
        #self.conv1 = nn.Conv2d(1, 64, 7)
        self.linear1_1 = nn.Linear(specsize, 1024)
        self.linear1_2 = nn.Linear(1024, 512)
        self.linear1_3 = nn.Linear(1024, 512)
        #self.linear1_3 = nn.Linear(256, 128)
        self.linear1_4 = nn.Linear(512, 256)
        
        self.lstm2_1 = nn.Linear(specsize, 1024)
        self.linear2_1 = nn.Linear(1024, 512)
        self.linear2_2 = nn.Linear(512, 256)
        #self.pool1 = nn.MaxPool2d(2)
        #self.conv2 = nn.Conv2d(64, 128, 5)
        #self.conv3 = nn.Conv2d(128, 256, 5)
        
        self.linear1 = nn.Linear(128, 2)
        self.linear2 = nn.Linear(64, 2)
        
        self.dropout = nn.Dropout(0.1)
        
        #self.linear2 = nn.Linear(512, 2)
        
    def forward(self, data):
        res = []
        for i in range(data.shape[1]):
            x = data[:, i]
            x = self.linear1_1(x.view(-1, specsize))
            x = F.relu(x)
            x = F.dropout(x, training=self.training)
            x = self.linear1_2(x)
            x = F.relu(x)
            x = F.normalize(x)
            #x = self.linear1_3(x)
            #x = F.relu(x)
            res.append(x)
        
#         x1 = data[:,0]
#         x1 = self.linear1_1(x1.view(-1, specsize))
#         x1 = F.relu(x1)
#         x1 = F.dropout(x1, training=self.training)
        
#         x1 = self.linear1_2(x1)
#         x1 = F.relu(x1)
        
#         x1 = self.linear1_3(x1)
#         x1 = F.relu(x1)
        
# #         x1 = self.linear1_3(x1)
# #         x1 = F.relu(x1)
        
# #         x1 = self.linear1_4(x1)
# #         x1 = F.relu(x1)
        
#         x2 = data[:,1]
        
#         x2 = self.linear1_1(x2.view(-1, specsize))
#         x2 = F.relu(x2)
#         x2 = F.dropout(x2, training=self.training)
        
#         x2 = self.linear1_2(x2)
#         x2 = F.relu(x2)
        
#         x2 = self.linear1_3(x2)
#         x2 = F.relu(x2)
        
#         x2 = self.linear1_3(x2)
#         x2 = F.relu(x2)
        
#         x2 = self.linear1_4(x2)
#         x2 = F.relu(x2)
        #res = F.pairwise_distance(x1, x2)
        #res = torch.abs(x1 - x2)
        #print(res.shape)
        #res = self.linear1(res)
        #res = self.linear2(res)
        
        #return F.log_softmax(res, 1)
        return res
        
#        for i in range(2): # Siamese nets; sharing weights
#            x = data[i]
#            x = self.conv1(x)
#            x = F.relu(x)
#            x = self.pool1(x)
#            x = self.conv2(x)
#            x = F.relu(x)
#            x = self.conv3(x)
#            x = F.relu(x)
#            
#            x = x.view(x.shape[0], -1)
#            x = self.linear1(x)
#            res.append(F.relu(x))
#        
#        res = torch.abs(res[1] - res[0])
#        res = self.linear2(res)
#        return res
    def name(self):
        return "Net"

In [51]:
for (tid, data) in enumerate(train_loader):
    if tid == 0:
        print(data[:,0:3:2].shape)
        break

torch.Size([1024, 2, 8000])


In [27]:
#hinge = torch.nn.HingeEmbeddingLoss()
triplet_loss = nn.TripletMarginLoss(margin=margin, p=2, reduction='sum')

def train(model, device, train_loader, epoch, optimizer):
    model.train()
    
    accurate_labels = 0
    all_labels = 0
    for (batch_idx, data) in enumerate(train_loader):
        #for i in range(len(data)):
        #    data[i] = data[i].to(device)
        
        data = data.to(device)    
        
        optimizer.zero_grad()
        
        #output_positive = model(data[:,0:2])
        #output_negative = model(data[:,0:3:2])
        Q, P, N = model(data)
#         target = target.type(torch.LongTensor).to(device)
        
#         target_positive = torch.squeeze(target[:,0])
#         target_negative = torch.squeeze(target[:,1])
        
        #change the labels back to [1, 0] for cross entropy
        #loss_positive = F.cross_entropy(output_positive, target_positive)
        #loss_negative = F.cross_entropy(output_negative, target_negative)
        #loss_positive = hinge(output_positive, target_positive)
        #loss_negative = hinge(output_negative, target_negative)
        #print('hinge distance: ' + str(output_positive - output_negative + margin))
        #dist_hinge = torch.clamp(output_positive - output_negative + margin, min=0.0)
        #loss = torch.sum(dist_hinge)
        loss = triplet_loss(Q, P, N)
        
        
        #loss = loss_positive + loss_negative
        loss.backward()
            
        optimizer.step()
        
        accurate_labels = accurate_labels + 2*torch.sum(F.pairwise_distance(Q, P)**2 - 
                                                        F.pairwise_distance(Q, N)**2 + margin < 0)
        #accurate_labels = accurate_labels + 2*torch.sum(output_positive + margin < output_negative)
        #accurate_labels_positive = torch.sum(torch.argmax(output_positive, dim=1) == target_positive).cpu()
        #accurate_labels_negative = torch.sum(torch.argmax(output_negative, dim=1) == target_negative).cpu()
        
        #accurate_labels = accurate_labels + accurate_labels_positive + accurate_labels_negative
        all_labels = all_labels + 2*len(Q)  
    
    accuracy = 100. * float(accurate_labels) / all_labels
    print('Epoch: ' + str(epoch))
    print('Train accuracy: {}/{} ({:.3f}%)'.format(accurate_labels, all_labels, accuracy))
    
def test(model, device, test_loader):
    model.eval()
    
    with torch.no_grad():
        accurate_labels = 0
        all_labels = 0
        loss = 0
        
        for batch_idx, (data, target) in enumerate(test_loader):
            #for i in range(len(data)):
            #    data[i] = data[i].to(device)
            
            data = data.to(device)    
        
            optimizer.zero_grad()

            #output_positive = model(data[:,0:2])
            #output_negative = model(data[:,0:3:2])
            Q, P = model(data)
#             target = target.type(torch.LongTensor).to(device)

#             target_positive = torch.squeeze(target[:,0])
#             target_negative = torch.squeeze(target[:,1])
            
            #loss_positive = F.cross_entropy(output_positive, target_positive)
            #loss_negative = F.cross_entropy(output_negative, target_negative)
            #loss_positive = hinge(output_positive, target_positive)
            #loss_negative = hinge(output_negative, target_negative)
            #dist_hinge = torch.clamp(dist_p - dist_n + self.margin, min=0.0)
            
            #loss = torch.max(loss_positive - loss_negative + 0.2, 0)#loss + loss_positive + loss_negative
            #dist_hinge = torch.clamp(output_positive - output_negative + margin, min=0.0)
            #loss = torch.sum(dist_hinge)
            #loss = triplet_loss(Q, P, N)
            
            #accurate_labels = accurate_labels + 2*torch.sum(F.pairwise_distance(Q, P)**2 - 
            #                                            F.pairwise_distance(Q, N)**2 + margin < 0)
            accurate_labels_positive = torch.sum(torch.argmax(output_positive, dim=1) == target_positive).cpu()
            #accurate_labels_negative = torch.sum(torch.argmax(output_negative, dim=1) == target_negative).cpu()
                
            #accurate_labels = accurate_labels + accurate_labels_positive + accurate_labels_negative
            #accurate_labels = accurate_labels + 2*torch.sum(output_positive + margin < output_negative)
            
            all_labels = all_labels + 2*len(Q)
                
        accuracy = 100. * float(accurate_labels) / all_labels
        print('Test accuracy: {}/{} ({:.3f}%)\tLoss: {:.6f}'.format(accurate_labels, all_labels, accuracy, loss))


In [28]:
def oneshot(model, device, data):
    model.eval()
    
    with torch.no_grad():
        for i in range(len(data)):
            data[i] = data[i].to(device)
            
        output = model(data)
        return torch.squeeze(torch.argmax(output, dim=1)).cpu().item()

In [29]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])

model = Net().to(device)
   
if do_learn: # training mode
    
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    #optimizer = optim.SGD(model.parameters(), lr=0.01)
    for epoch in range(num_epochs):
        train(model, device, train_loader, epoch, optimizer)
        test(model, device, test_loader)
        #if epoch & save_frequency == 0:
            #torch.save(model, 'siamese_{:03}.pt'.format(epoch))
        #else: # prediction
        #    prediction_loader = torch.utils.data.DataLoader(BalancedMNISTPair(
        #        '../data', train=False, download=True, transform=trans), batch_size=1, shuffle=True)
        #    model.load_state_dict(torch.load(load_model_path))
        #    data = []
        #    data.extend(next(iter(prediction_loader))[0][:3:2])
        #    same = oneshot(model, device, data)
        #    if same > 0:
        #        print('These two images are of the same number')
        #    else:
        #        print('These two images are not of the same number')
         

Epoch: 0
Train accuracy: 160796/208274 (77.204%)
Test accuracy: 50954/52070 (97.857%)	Loss: 5.848248
Epoch: 1
Train accuracy: 168488/208274 (80.897%)
Test accuracy: 51168/52070 (98.268%)	Loss: 4.305115
Epoch: 2
Train accuracy: 172652/208274 (82.897%)
Test accuracy: 51330/52070 (98.579%)	Loss: 3.260487
Epoch: 3
Train accuracy: 175962/208274 (84.486%)
Test accuracy: 51412/52070 (98.736%)	Loss: 2.443757
Epoch: 4
Train accuracy: 177948/208274 (85.439%)
Test accuracy: 51474/52070 (98.855%)	Loss: 2.195866
Epoch: 5
Train accuracy: 180364/208274 (86.599%)
Test accuracy: 51540/52070 (98.982%)	Loss: 1.792725
Epoch: 6
Train accuracy: 182408/208274 (87.581%)
Test accuracy: 51574/52070 (99.047%)	Loss: 1.709044
Epoch: 7
Train accuracy: 183822/208274 (88.260%)
Test accuracy: 51608/52070 (99.113%)	Loss: 1.621768
Epoch: 8
Train accuracy: 185578/208274 (89.103%)
Test accuracy: 51618/52070 (99.132%)	Loss: 1.624357
Epoch: 9
Train accuracy: 186886/208274 (89.731%)
Test accuracy: 51646/52070 (99.186%)	Loss:

Test accuracy: 51794/52070 (99.470%)	Loss: 0.463919
Epoch: 81
Train accuracy: 206600/208274 (99.196%)
Test accuracy: 51790/52070 (99.462%)	Loss: 0.388499
Epoch: 82
Train accuracy: 206690/208274 (99.239%)
Test accuracy: 51788/52070 (99.458%)	Loss: 0.336417
Epoch: 83
Train accuracy: 206632/208274 (99.212%)
Test accuracy: 51800/52070 (99.481%)	Loss: 0.364311
Epoch: 84
Train accuracy: 206670/208274 (99.230%)
Test accuracy: 51798/52070 (99.478%)	Loss: 0.318260
Epoch: 85
Train accuracy: 206752/208274 (99.269%)
Test accuracy: 51790/52070 (99.462%)	Loss: 0.360947
Epoch: 86
Train accuracy: 206710/208274 (99.249%)
Test accuracy: 51796/52070 (99.474%)	Loss: 0.413306
Epoch: 87
Train accuracy: 206792/208274 (99.288%)
Test accuracy: 51796/52070 (99.474%)	Loss: 0.385740
Epoch: 88
Train accuracy: 206752/208274 (99.269%)
Test accuracy: 51792/52070 (99.466%)	Loss: 0.360173
Epoch: 89
Train accuracy: 206902/208274 (99.341%)
Test accuracy: 51800/52070 (99.481%)	Loss: 0.382068
Epoch: 90
Train accuracy: 2068

Test accuracy: 51800/52070 (99.481%)	Loss: 0.395726
Epoch: 161
Train accuracy: 207818/208274 (99.781%)
Test accuracy: 51804/52070 (99.489%)	Loss: 0.369237
Epoch: 162
Train accuracy: 207730/208274 (99.739%)
Test accuracy: 51806/52070 (99.493%)	Loss: 0.409520
Epoch: 163
Train accuracy: 207800/208274 (99.772%)
Test accuracy: 51788/52070 (99.458%)	Loss: 0.429564
Epoch: 164
Train accuracy: 207780/208274 (99.763%)
Test accuracy: 51794/52070 (99.470%)	Loss: 0.372151
Epoch: 165
Train accuracy: 207894/208274 (99.818%)
Test accuracy: 51798/52070 (99.478%)	Loss: 0.353778
Epoch: 166
Train accuracy: 207792/208274 (99.769%)
Test accuracy: 51792/52070 (99.466%)	Loss: 0.350045
Epoch: 167
Train accuracy: 207754/208274 (99.750%)
Test accuracy: 51796/52070 (99.474%)	Loss: 0.364595
Epoch: 168
Train accuracy: 207828/208274 (99.786%)
Test accuracy: 51800/52070 (99.481%)	Loss: 0.329694
Epoch: 169
Train accuracy: 207840/208274 (99.792%)
Test accuracy: 51788/52070 (99.458%)	Loss: 0.298722
Epoch: 170
Train accu

In [ ]:
#torch.save(model, 'siamese99.1%modsworking.pt')

In [ ]:
remove modifications
use one charge

In [6]:
x = torch.randn((4, 3))
print(x)
x = F.normalize(x)
print(x)
print(torch.sqrt(torch.sum(x[0]**2)))

tensor([[ 0.0836, -0.2683, -2.6547],
        [-0.4704, -0.8377,  1.3105],
        [ 0.8315,  0.1661, -0.5774],
        [-0.8142,  0.5784, -1.6145]])
tensor([[ 0.0313, -0.1005, -0.9944],
        [-0.2895, -0.5155,  0.8065],
        [ 0.8105,  0.1619, -0.5629],
        [-0.4289,  0.3047, -0.8504]])
tensor(1.0000)
